<h1 align=center><font size = 5> K-Nearest Neighbors Classification in R: Exercise </font></h1>

### Exercise 1: Balance Scale

In this exercise, we will be using the <font color = "green">balance_scale.txt</font> data set. In this data set, each data point is classified as having the balance scale tip to the right, tip to the left, or be balanced. The variables of the data set are 'left weight', 'left distance', 'right weight', and 'right distance'. The class is determined by the greater of (right-distance x right-weight) and (left-distance x left-weight). However, if these values are equal, the data point is balanced.

The objective of this exercise is to use the k-nearest neighbors algorithm to determine the classification of test data and to estimate using knn regression, the difference between the product of right-distance and right-weight, and the product of left-distance and left-weight.

### a). Install the package

We first install and load the libraries that will be used in this exercise.

In [1]:
# Install the packages 'class' and 'kknn' and load their libraries, which will be needed for their k-nearest neighbors algorithms
install.packages("class")
install.packages("kknn")
library(class)
library(kknn)


  There is a binary version available but the source version is later:
      binary source needs_compilation
class 7.3-19 7.3-20              TRUE

  Binaries will be installed
package 'class' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\Erlangga\AppData\Local\Temp\Rtmp6FFseI\downloaded_packages


also installing the dependency 'igraph'




  There is a binary version available but the source version is later:
       binary source needs_compilation
igraph  1.2.6  1.3.5              TRUE

  Binaries will be installed
package 'igraph' successfully unpacked and MD5 sums checked
package 'kknn' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\Erlangga\AppData\Local\Temp\Rtmp6FFseI\downloaded_packages


Warning message:
"package 'class' was built under R version 3.6.3"Warning message:
"package 'kknn' was built under R version 3.6.3"

### b). Download the data set, load the data and view its structure

We can use the download.file command to download the data set, <font color = "green">"balance_scale.txt"</font>, load the data using the read.csv command, and view its structure using the str command. We will also look at the first few lines of the data using the head command.

In [2]:
# Load the data and view its structure
balance_scale <- read.csv("https://ibm.box.com/shared/static/684jzm7e6fbbssg87yc2v4dy53dgkdew.txt", sep = ",")
str(balance_scale)

# View the first few rows of the data using the head function
# Note: The raw data does not contain any column names
head(balance_scale)

'data.frame':	624 obs. of  5 variables:
 $ B   : Factor w/ 3 levels "B","L","R": 3 3 3 3 3 3 3 3 3 3 ...
 $ X1  : int  1 1 1 1 1 1 1 1 1 1 ...
 $ X1.1: int  1 1 1 1 1 1 1 1 1 1 ...
 $ X1.2: int  1 1 1 1 2 2 2 2 2 3 ...
 $ X1.3: int  2 3 4 5 1 2 3 4 5 1 ...


B,X1,X1.1,X1.2,X1.3
R,1,1,1,2
R,1,1,1,3
R,1,1,1,4
R,1,1,1,5
R,1,1,2,1
R,1,1,2,2


### c). Clean the data
Now we will clean the data by adding column names, as well new columns for the right and left products and their differences.

In [3]:
# Add column names
colnames(balance_scale) <- c("Class_Name","Left_Weight", "Left_Distance", "Right_Weight", "Right_Distance")
head(balance_scale)
# Note: We do not need to standardize the data in this instance since the numerical data values lie on the same scale.
# Calculate the products and differences
Right_Product <- balance_scale[,4]*balance_scale[,5]
Left_Product <- balance_scale[,2]*balance_scale[,3]
Differences <- Right_Product-Left_Product
# Add columns for Right_Product, Left_Product and Differences
balance_scale$Right_Product <- Right_Product
balance_scale$Left_Product <- Left_Product
balance_scale$Differences <- Differences

Class_Name,Left_Weight,Left_Distance,Right_Weight,Right_Distance
R,1,1,1,2
R,1,1,1,3
R,1,1,1,4
R,1,1,1,5
R,1,1,2,1
R,1,1,2,2


### d). Separate the data into training and test groups
Next we will split the data into two groups which will be used for training and testing respectively. To do this, we will create a vector of equal length to our data, with approximately 70% of the values in the vector being 1's and the remaining values being 2's. The order in which the values appear will be random and depending on the indices of the vector which correspond to the value of 1 will be used to select our training data, while the indices corresponding to 2 will be used to select our test data.

In [4]:
# Use the sample function to create a vector of indices that will be used
set.seed(1234)
ind <- sample(2, nrow(balance_scale), replace=TRUE, prob=c(0.7, 0.3))

# Create the training and test data from the dataset using ind
bscale.train <- balance_scale[ind==1, 6:8]
bscale.test <- balance_scale[ind==2, 6:8]

# Create the target vectors for the training and test data from the dataset using ind
bscale.trainLabels <- balance_scale[ind==1, 1]
bscale.testLabels <- balance_scale[ind==2, 1]

### e). Implementation of k-nearest neighbors for classification
Now we will use the knn algorithm from the 'class' package to perform our classification task. This will be done using the knn command to determine the class name (R,L,B) of our test data based on our training data and our choice of k.

In [5]:
# Use the knn command to make predictions on the Class_Name of the test data
knn_class <- knn(train = bscale.train, test = bscale.test, cl = bscale.trainLabels, k=3)

# Find the number of incorrectly classified points
correct <- which(knn_class == bscale.testLabels, arr.ind = TRUE)
incorrect <- which(knn_class != bscale.testLabels, arr.ind = TRUE)
cat("Number of incorrectly classified points:",length(incorrect),"\n")

# Find the proportion of correctly classified points
proportion_correct <- length(correct)/length(bscale.testLabels)
cat("Proportion of correctly classified points", proportion_correct,"\n")

Number of incorrectly classified points: 1 
Proportion of correctly classified points 0.9947368 


### f). K-nearest neighbors regression using the kknn command
The idea of using regression is by far predicting the continious response variable, instead of class variables, here the method is similar to localized linear regressions a.k.a LOESS, technique used, where $k$ is equivalent to the spread of kernel function around the point. [Reference](http://127.0.0.1:44104/help/library/kknn/doc/paper399.pdf) is good starting point to know about this methodology 

In [6]:
# Run the knn regression using the kknn command
knn_reg <- kknn(formula = Differences ~ ., train=bscale.train, test=bscale.test, k=3)

# Find the number of incorrectly classified points
incorrect_reg <- which(knn_reg$fitted.values != bscale.test$Differences, arr.ind = TRUE)
cat("Number of incorrectly classified points:", length(incorrect_reg), "\n");

# Find the proportion of correctly classified points
correct_reg <- which(knn_reg$fitted.values == bscale.test$Differences, arr.ind = TRUE)
cat("Proportion of correctly classified points", length(correct_reg)/length(bscale.test$Differences), "\n")


# Display the first few rows of the regression estimates of the differences and their true values
head(cbind(knn_reg$fitted.values,bscale.test$Differences))

Number of incorrectly classified points: 134 
Proportion of correctly classified points 0.2947368 


0.5555556,1
13.5555556,14
6.5555556,7
0.0000000,0
2.1111111,2
3.0000000,3


We can also determine the optimal value of k using the train.kknn command.

In [7]:
best_reg <- train.kknn(formula = Differences ~ ., data=bscale.train, kmax=8)
best_reg$best.parameters  

$kernel
[1] "optimal"

$k
[1] 2

The results indicate that the best value for k, in terms of knn regression, is k=2. This value makes sense given that the k-nearest neighbors regression algorithm estimates the outcome using a weighted average of the k nearest neighbors, weighted by the inverse of their distance.

In [8]:
# Run the knn regression again using the kknn command with k=2
knn_reg2 <- kknn(formula = Differences ~ ., train=bscale.train, test=bscale.test, k=2)

# Find the number of incorrectly classified points
incorrect_reg2 <- which(knn_reg2$fitted.values != bscale.test$Differences, arr.ind = TRUE)
cat("Number of incorrectly classified points:", length(incorrect_reg2),"\n")
# Find the proportion of correctly classified points
correct_reg2 <- which(knn_reg2$fitted.values == bscale.test$Differences, arr.ind = TRUE)
cat("Proportion of correctly classified points",length(correct_reg2)/length(bscale.test$Differences),"\n")

# Display the first few rows of the new regression estimates of the differences and their true values
head(cbind(knn_reg2$fitted.values,bscale.test$Differences))

Number of incorrectly classified points: 81 
Proportion of correctly classified points 0.5736842 


0.75,1
13.75,14
6.75,7
0.00,0
2.00,2
3.00,3


As can be seen, the estimates of the regression using k = 2 are closer to the true values with almost double the accuracy.

### g). K-nearest neighbors classification using the kknn command

We can also use the kknn command for classification:

In [9]:
# The kknn function can also be used for classification
knn_class2 <- kknn(formula = Class_Name ~ ., train=subset(balance_scale, select=c(Class_Name,Right_Product,Left_Product,Differences))[ind==1,], test=subset(balance_scale, select=c(Class_Name,Right_Product,Left_Product,Differences))[ind==2,], k=3)
# Find the number of incorrectly classified points
incorrect_class2 <- which(knn_class2$fitted.values != bscale.testLabels, arr.ind = TRUE)
cat("Number of incorrectly classified points:",length(incorrect_class2),"\n")

Number of incorrectly classified points: 1 


Now let's try finding the optimal value of k for classification.

In [10]:
best_class <- train.kknn(formula = Class_Name ~ ., data=subset(balance_scale, select=c(Class_Name,Right_Product,Left_Product,Differences))[ind==1,], kmax=8)
best_class$best.parameters

$kernel
[1] "optimal"

$k
[1] 1

In [11]:
# Using k=1
knn_class3 <- kknn(formula = Class_Name ~ ., train=subset(balance_scale, select=c(Class_Name,Right_Product,Left_Product,Differences))[ind==1,], test=subset(balance_scale, select=c(Class_Name,Right_Product,Left_Product,Differences))[ind==2,], k=1)
# Find the number of incorrectly classified points
incorrect_class3 <- which(knn_class3$fitted.values != bscale.testLabels, arr.ind = TRUE)
cat("Number of incorrectly classified points:", length(incorrect_class3),"\n")

Number of incorrectly classified points: 1 


The results of using k=1 are apparently similar to those of using k=3.

   ### Exercise 2: Second Language Immersion Schools in Canada (Classification)

French immersion programs are designed for students who are not native French speakers or students who do not speak French at home. These programs provide instruction in the classroom entirely in French until the end of grade 3, although certain specialist subjects may be instructed in English. We are provided with a data set of Canadian schools offering French immersion programs (<font color = "green">"Second_Language_Immersion_Schools_in_Canada.csv"</font>), which contains information regarding these schools such as their names, provinces and coordinates.

Suppose we would like to determine the province in which a certain school or other place of interest is located in Canada, given that we have the coordinates (latitude and longitude) of this place. We can do this by using the k-nearest neighbor algorithm.


### a). Download the data set, load the data and view its structure

We can use the download.file command to download the data set, <font color = "green">"Second_Language_Immersion_Schools_in_Canada.csv"</font>, load the data using the read.csv command, and view its structure using the str command.

In [12]:
# Load the data and view the structure
schools <- read.csv("https://ibm.box.com/shared/static/uummw8ijp41gn3nfkuipi78xnalkss4c.csv", sep = ",")
str(schools)

'data.frame':	1973 obs. of  24 variables:
 $ source.code                               : Factor w/ 1973 levels "10_100_004","10_100_005",..: 1720 1722 1718 1719 1723 1721 1921 1923 1922 367 ...
 $ name                                      : Factor w/ 1934 levels "‚cole Centennial School",..: 398 424 444 1079 1444 1849 182 183 639 647 ...
 $ Canadian.Heritage.region.code             : Factor w/ 4 levels "pch01","pch03",..: 4 4 4 4 4 4 4 4 4 2 ...
 $ Canadian.Heritage.region.name..english    : Factor w/ 4 levels "Atlantic Region",..: 4 4 4 4 4 4 4 4 4 2 ...
 $ Canadian.Heritage.region.name..french     : Factor w/ 4 levels "Ontario","R‚gion de l'Atlantique",..: 3 3 3 3 3 3 3 3 3 1 ...
 $ province.code                             : Factor w/ 10 levels "pr10","pr11",..: 9 9 9 9 9 9 9 9 9 5 ...
 $ province.name..english                    : Factor w/ 10 levels "Alberta","British Columbia",..: 2 2 2 2 2 2 2 2 2 8 ...
 $ province.name..french                     : Factor w/ 10 levels "?le-du-P

### b). Clean the data
For the task we would like to do we only need a few variables of the data. These variables are school name (just for viewing), province, latitude and longitude. We will also change the variable names to cleaner names where possible.

In [13]:
# Create a subset of the data containing the variables province, latitude, and longitude
schools.sub <- subset(schools, select=c(name, province.name..english, latitude, longitude))
head(schools.sub)
# Change the column names to cleaner names
colnames(schools.sub) <- c("name", "province", "latitude", "longitude")
head(schools.sub)

name,province.name..english,latitude,longitude
Centennial Park Elementary,British Columbia,49.04759,-122.3154
Chief Dan George Middle,British Columbia,49.06641,-122.3137
Clearbrook Elementary,British Columbia,49.06815,-122.3380
Margaret Stenersen Elementary,British Columbia,49.05723,-122.2706
Sandy Hill Elementary,British Columbia,49.07144,-122.2493
W J Mouat Secondary,British Columbia,49.05892,-122.3290


name,province,latitude,longitude
Centennial Park Elementary,British Columbia,49.04759,-122.3154
Chief Dan George Middle,British Columbia,49.06641,-122.3137
Clearbrook Elementary,British Columbia,49.06815,-122.3380
Margaret Stenersen Elementary,British Columbia,49.05723,-122.2706
Sandy Hill Elementary,British Columbia,49.07144,-122.2493
W J Mouat Secondary,British Columbia,49.05892,-122.3290


### c). Visualize the data on a map
We will now install and load packages that will enable us to visualize our data by displayiing it on a map. This should make it easier to see clearly the predictions made by the knn algorithm.

In [14]:
# Installing leaflet maps
install.packages("leaflet")
library(leaflet)
# Packages used to display the maps in this notebook
library(htmlwidgets)
library(IRdisplay)

also installing the dependencies 'rlang', 'fastmap', 'htmltools', 'terra', 'viridisLite', 'gridExtra', 'crosstalk', 'htmlwidgets', 'png', 'raster', 'sp', 'viridis', 'leaflet.providers'




  There are binary versions available but the source versions are later:
             binary source needs_compilation
rlang        0.4.11  1.0.6              TRUE
htmltools   0.5.1.1  0.5.4              TRUE
terra         1.2-5 1.6-47              TRUE
viridisLite   0.4.0  0.4.1             FALSE
crosstalk     1.1.1  1.2.0             FALSE
htmlwidgets   1.5.3  1.6.1             FALSE
png           0.1-7  0.1-8              TRUE
raster       3.4-10 3.6-13              TRUE
sp            1.4-5  1.5-1              TRUE
viridis       0.6.1  0.6.2             FALSE
leaflet     2.0.4.1  2.1.1             FALSE

  Binaries will be installed
package 'rlang' successfully unpacked and MD5 sums checked
package 'fastmap' successfully unpacked and MD5 sums checked
package 'htmltools' successfully unpacked and MD5 sums checked


Warning message:
"cannot remove prior installation of package 'htmltools'"Warning message in file.copy(savedcopy, lib, recursive = TRUE):
"problem copying C:\Users\Erlangga\anaconda3\envs\RKernel\Lib\R\library\00LOCK\htmltools\libs\x64\htmltools.dll to C:\Users\Erlangga\anaconda3\envs\RKernel\Lib\R\library\htmltools\libs\x64\htmltools.dll: Permission denied"Warning message:
"restored 'htmltools'"

package 'terra' successfully unpacked and MD5 sums checked
package 'gridExtra' successfully unpacked and MD5 sums checked
package 'png' successfully unpacked and MD5 sums checked
package 'raster' successfully unpacked and MD5 sums checked
package 'sp' successfully unpacked and MD5 sums checked
package 'leaflet.providers' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\Erlangga\AppData\Local\Temp\Rtmp6FFseI\downloaded_packages


installing the source packages 'viridisLite', 'crosstalk', 'htmlwidgets', 'viridis', 'leaflet'

Warning message in install.packages("leaflet"):
"installation of package 'htmlwidgets' had non-zero exit status"Warning message in install.packages("leaflet"):
"installation of package 'leaflet' had non-zero exit status"

ERROR: Error in library(leaflet): there is no package called 'leaflet'


Next we will set up a default parameters for the setView attribute of the map we are creating.

In [15]:
# Establish the limits of our default visualization
lower_lon = -140
upper_lon = -50
lower_lat = 40
upper_lat = 65
# Establish the center of our default visualization
center_lon = (lower_lon + upper_lon)/2
center_lat = (lower_lat + upper_lat)/2
# Set the zoom of our default visualization
zoom = 4

Now we create and display our map

In [16]:
# Create a leaflet map
schools_map <- leaflet(schools.sub) %>%
  setView(center_lon,center_lat, zoom)%>% 
  addProviderTiles("OpenStreetMap.BlackAndWhite")%>% # set the map that we want to use as background
  addCircleMarkers(lng = schools.sub$longitude, 
                   lat = schools.sub$latitude, 
                   popup = schools.sub$name, # pop-ups will show the name of school if you click on a data point
                   fillColor = "Black", # colors of the markers will be black
                   fillOpacity = 1, # the shapes will have maximum opacity
                   radius = 4, # radius determine the size of each shape
                   stroke = F) # no stroke will be drawn in each data point

saveWidget(schools_map, file="schools_map.html", selfcontained = F) #saving the leaflet map in html
display_html(paste("<iframe src=' ", 'schools_map.html', " ' width='100%' height='400'","/>")) #display the map !


ERROR: Error in leaflet(schools.sub) %>% setView(center_lon, center_lat, zoom) %>% : could not find function "%>%"


### d). Implementation of k-nearest neighbors
Now we will use the knn algorithm from the 'class' package to perform our classification task. This will be done by first separating our data into training and test groups, selecting our k parameter value, and then using them to make predictions of the provinces of our test data using the knn command.

In [17]:
# Use the sample function to create a vector of indices that will be used to split our data into training and test data
set.seed(1234)
ind <- sample(2, nrow(schools.sub), replace=TRUE, prob=c(0.7, 0.3)) # This creates a vector of equal length to our data, with approximately 70% of the values being 1 and the remaining values are 2  
# Create training and test sets containing only the geographical locations 
schools.train <- schools.sub[ind==1,3:4]
schools.test <- schools.sub[ind==2,3:4]
# Note: Normalizing or standardizing the geographical attributes of the data will not be helpful since values are already on the same scale 
# Create the target vectors for the training and test data from the original dataset using ind
schools.trainLabels <- schools.sub[ind==1,2]
schools.testLabels <- schools.sub[ind==2,2]
# Run the knn algorithm to classify test data points into the different provinces
prov_pred <- knn(train = schools.train, test = schools.test, cl = schools.trainLabels, k=3)

# Find the number of incorrectly classified points
correct_provinces <- which(prov_pred == schools.testLabels, arr.ind = TRUE)
incorrect_provinces <- which(prov_pred != schools.testLabels, arr.ind = TRUE)
cat("Number of incorrectly classified points:", length(incorrect_provinces),"\n")
# Find the proportion of correctly classified points
proportion_of_correct_provinces <- length(correct_provinces)/length(schools.testLabels)
cat("Proportion of correctly classified points", proportion_of_correct_provinces,"\n")

Number of incorrectly classified points: 6 
Proportion of correctly classified points 0.9893428 


### e). Plot the classification predictions of k-nearest neighbors
Now we will link the knn predictions of the provinces with the coordinates of the test data, and display them on a map.

In [18]:
# Link the knn predictions with the test data
schools.test$prov <- prov_pred
# Plot the test data with predicted provinces
color <- colorFactor(topo.colors(10), schools.test$prov)
schools_map2 <- leaflet(schools.test) %>%
  setView(center_lon,center_lat, zoom)%>% 
  addProviderTiles("OpenStreetMap.BlackAndWhite")%>% # set the map that we want to use as background
  addCircleMarkers(lng = schools.test$longitude, 
                   lat = schools.test$latitude, 
                   popup = schools.test$prov, # pop-ups will show the predicted province if you click on a data point
                   fillColor = ~color(prov),
                   fillOpacity = 1, # the shapes will have maximum opacity
                   radius = 4, # radius determine the size of each shape
                   stroke = F) # no stroke will be drawn in each data point

saveWidget(schools_map2, file="schools_map2.html", selfcontained = F) #saving the leaflet map in html
display_html(paste("<iframe src=' ", 'schools_map2.html', " ' width='100%' height='400'","/>")) #display the map !



ERROR: Error in colorFactor(topo.colors(10), schools.test$prov): could not find function "colorFactor"
